In [1]:
import pandas as pd
df = pd.read_csv('musinsa_data.csv')
df.head()

,Unnamed: 0,temp,c_number,c_name,gender,height(cm),weight(kg),fit,link,size,c_result
0,0,28,0,[패키지] 오버핏 쿨 마이크로아이스 슬리브리스 2P SET 6COLOR,남자,174cm,66kg,적당함,//image.msscdn.net/images/goods_img/20200528/1...,L,L0
1,1,28,0,[패키지] 오버핏 쿨 마이크로아이스 슬리브리스 2P SET 6COLOR,남자,175cm,90kg,적당함,//image.msscdn.net/images/goods_img/20200528/1...,XXL,XXL0
2,2,28,0,[패키지] 오버핏 쿨 마이크로아이스 슬리브리스 2P SET 6COLOR,남자,180cm,75kg,적당함,//image.msscdn.net/images/goods_img/20200528/1...,L,L0
3,3,28,0,[패키지] 오버핏 쿨 마이크로아이스 슬리브리스 2P SET 6COLOR,남자,170cm,80kg,적당함,//image.msscdn.net/images/goods_img/20200528/1...,L,L0
4,4,28,0,[패키지] 오버핏 쿨 마이크로아이스 슬리브리스 2P SET 6COLOR,남자,180cm,83kg,적당함,//image.msscdn.net/images/goods_img/20200528/1...,XL,XL0


### height,weight 에서 cm, kg 제거 

In [4]:
df['height(cm)'] = df['height(cm)'].str.replace('cm', '').astype(int)
df['weight(kg)'] = df['weight(kg)'].str.replace('kg', '').astype(int)

### 필요없는 행 지우기 ( Unnamed: 0,c_name,link

In [42]:
df.drop(labels=['size'],axis=1,inplace=True)
df

,temp,c_number,gender,height(cm),weight(kg),fit,c_result
0,28,0,남자,174,66,적당함,L0
1,28,0,남자,175,90,적당함,XXL0
2,28,0,남자,180,75,적당함,L0
3,28,0,남자,170,80,적당함,L0
4,28,0,남자,180,83,적당함,XL0
...,...,...,...,...,...,...,...
6147,4,39,여자,160,42,큼,M39
6148,4,39,남자,179,73,적당함,L39
6149,4,39,남자,184,97,적당함,XL39
6150,4,39,여자,180,57,적당함,XL39


### 원핫 인코딩 
- gender ( 남자 / 여자 ) 
- fit ( 적당함/ 큼 / 여유 ) 

In [43]:
df_encoded = pd.get_dummies(df,columns=['gender','fit'])
df_encoded.head()


,temp,c_number,height(cm),weight(kg),c_result,gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
0,28,0,174,66,L0,1,0,0,1,0
1,28,0,175,90,XXL0,1,0,0,1,0
2,28,0,180,75,L0,1,0,0,1,0
3,28,0,170,80,L0,1,0,0,1,0
4,28,0,180,83,XL0,1,0,0,1,0


### 스케일링

In [45]:
from sklearn.preprocessing import StandardScaler


cols_to_scale = ['temp', 'c_number', 'height(cm)', 'weight(kg)']
scaler = StandardScaler()

df_encoded[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])

df_encoded.head()


,temp,c_number,height(cm),weight(kg),c_result,gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
0,1.652407,-1.821266,0.278038,-0.032816,L0,1,0,0,1,0
1,1.652407,-1.821266,0.404107,1.952906,XXL0,1,0,0,1,0
2,1.652407,-1.821266,1.034447,0.711830,L0,1,0,0,1,0
3,1.652407,-1.821266,-0.226234,1.125522,L0,1,0,0,1,0
4,1.652407,-1.821266,1.034447,1.373737,XL0,1,0,0,1,0


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


### 훈련 세트, 테스트 세트 생성 , 훈련 및 성능 평가 

In [49]:
X = df_encoded.drop('c_result', axis=1)
y = df_encoded['c_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle = True)



### SVC

In [50]:
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
cancer = load_breast_cancer()

model = SVC(gamma=0.01).fit(X_train,y_train)

print("훈련세트 정확도:{:.2f}".format(model.score(X_train,y_train)))
print("테스트세트 정확도:{:.2f}".format(model.score(X_test,y_test)))

훈련세트 정확도:0.14
테스트세트 정확도:0.13


### SMOTE 오버샘플링

In [59]:
# from sklearn.svm import SVC
# from imblearn.over_sampling import SMOTE
# from sklearn.metrics import accuracy_score

# smote = SMOTE(sampling_strategy='auto', k_neighbors=1, random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# model = SVC(gamma=0.01).fit(X_train_resampled,y_train_resampled)

# print("훈련세트 정확도:{:.2f}".format(model.score(X_train_resampled,X_train_resampled)))
# # print("테스트세트 정확도:{:.2f}".format(model.score(X_test,y_test)))

In [62]:
!pip install imbalanced-learn

In [94]:
X = df_encoded.drop('c_result', axis=1)
y = df_encoded['c_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle = True)


In [95]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

In [81]:
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [82]:
y_pred = random_forest_model.predict(X_test)

In [83]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.323146944083225


In [76]:
df_encoded.drop(labels=['c_number'],axis=1,inplace=True)

KeyError: "['c_number'] not found in axis"

In [71]:
user_input = [22,1,177,70,]

4928    M29
3347    M21
1359    M10
6146    M39
4506    S24
       ... 
3772    L21
5191    M28
5226    L29
5390    M31
860      M5
Name: c_result, Length: 4614, dtype: object

In [87]:
X_train

,temp,height(cm),weight(kg),gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
4928,-0.965638,-1.739052,-1.935798,0,1,0,0,1
3347,0.081580,-0.856575,-0.115554,1,0,0,1,0
1359,0.866994,-1.739052,-0.529246,0,1,0,1,0
6146,-1.489248,0.025902,-0.115554,1,0,0,1,0
4506,-0.834736,-1.108711,-1.604845,0,1,0,1,0
...,...,...,...,...,...,...,...,...
3772,-0.442029,1.412652,0.794568,1,0,0,1,0
5191,-1.358345,0.278038,-0.115554,1,0,0,1,0
5226,-1.358345,0.530175,0.132661,1,0,0,1,0
5390,-1.358345,-0.226234,-0.611984,1,0,0,1,0


In [84]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [85]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [86]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.44213263979193757


In [88]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [89]:
X_train

,temp,height(cm),weight(kg),gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
2481,0.212482,1.034447,0.794568,1,0,0,1,0
5325,-1.358345,0.908379,0.132661,1,0,1,0,0
443,1.652407,-1.486916,-0.363769,0,1,0,1,0
4022,-0.572932,-0.226234,0.546353,1,0,0,1,0
5253,-1.358345,0.530175,-0.032816,1,0,0,1,0
...,...,...,...,...,...,...,...,...
3772,-0.442029,1.412652,0.794568,1,0,0,1,0
5191,-1.358345,0.278038,-0.115554,1,0,0,1,0
5226,-1.358345,0.530175,0.132661,1,0,0,1,0
5390,-1.358345,-0.226234,-0.611984,1,0,0,1,0


In [91]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [92]:
clf = DecisionTreeClassifier()
clf.fit(X_resampled, y_resampled)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.16896831844029245


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_resampled, y_resampled)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.16896831844029245


In [96]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

clf.fit(X_train_resampled, y_train_resampled)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3953185955786736
